<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Get-select-symbols-in-single-df" data-toc-modified-id="Get-select-symbols-in-single-df-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Get select symbols in single df</a></span></li></ul></div>

In [2]:
%load_ext autoreload
%autoreload 2

%load_ext watermark

from pathlib import PurePath, Path
from collections import OrderedDict as od
import sys
import time 
import os
import json
from pprint import pprint, pformat

import pandas as pd
import numpy as np
import pymc3 as pm
from dask.diagnostics import ProgressBar
pbar = ProgressBar()
pbar.register()
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import numba as nb
import pystore

# import visual tools
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
import seaborn as sns

sns_params = {
    'font.size':9.5,
    'font.weight':'medium',
    'figure.figsize':(10,7),
}

plt.style.use('seaborn-talk')
plt.style.use('bmh')
sns.set_context(sns_params)
#plt.rcParams['font.family'] = 'DejaVu Sans Mono'
#plt.rcParams['font.size'] = 9.5
#plt.rcParams['font.weight'] = 'medium'
#plt.rcParams['figure.figsize'] = 10,7
#blue, green, red, purple, gold, teal = sns.color_palette('colorblind', 6)

# import util libs
from tqdm import tqdm, tqdm_notebook
import warnings
warnings.filterwarnings("ignore")

from dotenv import load_dotenv
load_dotenv(verbose=True)

from logzero import logger as lz_logger

from src.tools.pystore_tools import *
from src.tools.utils import *
from src.CONSTANTS import *

pdir = get_relative_project_dir(
    project_repo_name=PROJECT_REPO_NAME, partial=False)
data_dir = Path(pdir/'data')

%watermark -v -m -g
print()
%watermark --iversions

Can not use cuDNN on context None: cannot compile with cuDNN. We got this error:
b'In file included from /tmp/try_flags_rhuhvzcq.c:3:0:\n/usr/include/stdio.h:27:10: fatal error: bits/libc-header-start.h: No such file or directory\n #include <bits/libc-header-start.h>\n          ^~~~~~~~~~~~~~~~~~~~~~~~~~\ncompilation terminated.\n'
Mapped name None to device cuda: GeForce GTX 980 Ti (0000:01:00.0)


CPython 3.7.2
IPython 6.5.0

compiler   : GCC 7.3.0
system     : Linux
release    : 4.19.11-041911-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 12
interpreter: 64bit
Git hash   :

json        2.0.9
numpy       1.15.4
pandas      0.23.4
pymc3       3.6
numba       0.41.0
pystore     0.1.5
matplotlib  3.0.2
seaborn     0.9.0
dask        1.0.0
logzero     1.5.0



In [3]:
data_ext = data_dir / 'external'
symbols = (pd.read_csv(
    Path(data_ext/'ETFList.Options.Nasdaq__M.csv')
))#.Symbol)
symbols[:5]

,Symbol,Name,LastSale,NetChange,NetChangeDirection,PercentChange,1YrPercentChange
0,AAXJ,iShares MSCI All Country Asia ex Japan Index Fund,54.7100,-0.2119,down,0.39,2.76
1,ACWI,iShares MSCI ACWI Index Fund,59.4800,-0.0868,down,0.15,8.21
2,ACWV,iShares Edge MSCI Min Vol Global ETF,72.4498,-1.2402,down,1.68,6.75
3,ACWX,iShares MSCI ACWI ex US Index Fund,40.1600,-0.0401,down,0.10,3.10
4,ADRE,BLDRS Emerging Markets 50 ADR Index Fund,32.2600,0.0300,up,0.09,10.34


In [4]:
pystore_data_dir = Path('/media/bcr/HDD/Code_Backups_Sync/iex_intraday_equity_downloader/data/processed')
item = get_item(symbols.Symbol[0], pystore_data_dir)

In [5]:
keep_cols = ['lastSalePrice','lastSaleSize','volume','spread','mid_price','dollar_volume']
df = item.data.loc['2018-10-01':'2018-12-31'][keep_cols].resample('1Min').mean().dropna().compute()
cprint(df)

[########################################] | 100% Completed |  1.6s
-------------------------------------------------------------------------------
dataframe information
-------------------------------------------------------------------------------
                     lastSalePrice  lastSaleSize         volume    spread  \
index                                                                       
2018-12-31 13:55:00      63.971667    605.000000  388675.666667  0.063333   
2018-12-31 13:56:00      63.990000    400.000000  389254.000000  0.060000   
2018-12-31 13:57:00      63.905000    962.000000  390854.000000  0.050000   
2018-12-31 13:58:00      63.900000    266.666667  391662.000000  0.053333   
2018-12-31 13:59:00      63.936667    166.666667  391828.666667  0.023333   

                     mid_price  dollar_volume  
index                                          
2018-12-31 13:55:00  63.965000   2.486425e+07  
2018-12-31 13:56:00  64.000000   2.490836e+07  
2018-12-31 13:57:0

In [13]:
issuers = ['Vanguard'] #['iShares','SPDR','Vanguard']#,'PowerShares','Schwab','JPMorgan','Guggenheim','First Trust']
names = [x for x in symbols['Name'] if any(y in x for y in issuers)]
select_symbol_df = symbols[symbols['Name'].isin(names)]
select_symbols = select_symbol_df.Symbol
print(select_symbols.shape)

print(select_symbol_df.Name.sort_values())

(42,)

## Get select symbols in single df

In [7]:
error_symbols = []
D = dict()

keep_cols = ['lastSalePrice','lastSaleSize','volume',
             'spread','mid_price','dollar_volume']
start, end = '2018-10-01', '2018-12-31'

for sym in tqdm_notebook(select_symbols):
    print(f'********* {sym} *********')
    item = get_item(sym, pystore_data_dir)
    try:
        tmp_df = (item.data
                  .loc[start:end]
                  [keep_cols]
                  .resample('1Min')
                  .mean()
                  .dropna()
                  .compute())
        D[sym] = tmp_df  
    except Exception as e:
        print(f'{sym} error: {e}')
        try:
            tmp_df = (item.data
                      .loc[start:end]
                      [keep_cols]
                      .compute()
                      .resample('1Min')
                      .mean()
                      .dropna())
            D[sym] = tmp_df
        except Exception as e:
            print(f'{sym} second error: {e}')
            error_symbols.append(sym)
            continue
            
print(error_symbols)            

********* AAXJ *********
[########################################] | 100% Completed |  0.3s
********* ACWI *********
[########################################] | 100% Completed |  0.3s
********* ACWV *********
[########################################] | 100% Completed |  0.3s
********* ACWX *********
[########################################] | 100% Completed |  0.3s
********* AGG *********
[########################################] | 100% Completed |  0.3s
********* BIL *********
[########################################] | 100% Completed |  0.3s
********* BIV *********
[########################################] | 100% Completed |  0.3s
********* BKF *********
[########################################] | 100% Completed |  0.2s
********* BLV *********
[########################################] | 100% Completed |  0.2s
********* BND *********
[########################################] | 100% Completed |  0.3s
********* BNDX *********
[########################################] | 100% C

[########################################] | 100% Completed |  0.2s
********* HEWJ *********
[########################################] | 100% Completed |  0.3s
********* HEZU *********
[########################################] | 100% Completed |  0.3s
********* HYG *********
[########################################] | 100% Completed |  0.3s
********* HYMB *********
[########################################] | 100% Completed |  0.2s
********* IAI *********
[########################################] | 100% Completed |  0.2s
********* IAT *********
[########################################] | 100% Completed |  0.2s
********* IBB *********
[########################################] | 100% Completed |  0.3s
********* ICF *********
[########################################] | 100% Completed |  0.2s
********* IDU *********
[########################################] | 100% Completed |  0.2s
********* IDV *********
[########################################] | 100% Completed |  0.3s
*********

[########################################] | 100% Completed |  0.3s
********* PFF *********
[########################################] | 100% Completed |  0.3s
********* QUAL *********
[########################################] | 100% Completed |  0.3s
********* REM *********
[########################################] | 100% Completed |  0.2s
********* REZ *********
[########################################] | 100% Completed |  0.1s
********* RING *********
[########################################] | 100% Completed |  0.2s
********* RWO *********
[########################################] | 100% Completed |  0.2s
********* RWR *********
[########################################] | 100% Completed |  0.2s
********* RWX *********
[########################################] | 100% Completed |  0.2s
********* SCJ *********
[########################################] | 100% Completed |  0.2s
********* SCPB *********
SCPB error: Can only resample dataframes with known divisions
See https://doc

In [8]:
df = (pd.concat(D, keys=D.keys()))
df.index.names = ['symbol','dates']
cprint(df)

-------------------------------------------------------------------------------
dataframe information
-------------------------------------------------------------------------------
                            lastSalePrice  lastSaleSize  volume  spread  \
symbol dates                                                              
XSD    2018-12-31 13:55:00           64.6         100.0   200.0     0.0   
       2018-12-31 13:56:00           64.6         100.0   200.0     0.0   
       2018-12-31 13:57:00           64.6         100.0   200.0     0.0   
       2018-12-31 13:58:00           64.6         100.0   200.0     0.0   
       2018-12-31 13:59:00           64.6         100.0   200.0     0.0   

                            mid_price  dollar_volume  
symbol dates                                          
XSD    2018-12-31 13:55:00        0.0        12920.0  
       2018-12-31 13:56:00        0.0        12920.0  
       2018-12-31 13:57:00        0.0        12920.0  
       2018-12-31

In [9]:
outfp = Path(data_dir/'processed'/'etf_symbols_01.parq')
df.to_parquet(outfp)